In [1]:
from functools import partial
from torch.utils.data import DataLoader

from problems import ContinuousQuadratic, DiscreteQuadratic
from src.data import OfflineDataset, OnlineDataset, custom_collate_fn
from src.model import DPT

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
n = 1
d = 1

if n == 1:
    problem_class = ContinuousQuadratic
    problems = [problem_class(d=d, n=n, seed=seed) for seed in range(10)]
else:
    problem_class = DiscreteQuadratic
    problems = [problem_class(d=d, n=n, seed=seed) for seed in range(10)]

In [12]:
seq_len = 3
batch_size = 2

dataset = OfflineDataset(problems, seq_len=seq_len)
collate_fn = partial(custom_collate_fn, problem_class=problem_class)
dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)
batch = next(iter(dataloader))
batch

{'x': tensor([[[0.5488],
          [0.7152],
          [0.6028]],
 
         [[0.5488],
          [0.7152],
          [0.6028]]]),
 'y': tensor([[0.4171, 0.4739, 0.4002],
         [0.9162, 1.9833, 1.2314]]),
 'x_min': tensor([[6.0276e-01],
         [1.1437e-04]]),
 'problem': [<problems.quadratic.ContinuousQuadratic at 0x7baa833d03e0>,
  <problems.quadratic.ContinuousQuadratic at 0x7baaf0413c80>]}

In [13]:
print('x', batch["x"].shape, batch["x"].dtype)
print('y', batch["y"].shape, batch["y"].dtype)
print('x_min', batch["x_min"].shape, batch["x_min"].dtype)
print('problem', batch["problem"], type(batch["problem"]))

x torch.Size([2, 3, 1]) torch.float32
y torch.Size([2, 3]) torch.float32
x_min torch.Size([2, 1]) torch.float32
problem [<problems.quadratic.ContinuousQuadratic object at 0x7baa833d03e0>, <problems.quadratic.ContinuousQuadratic object at 0x7baaf0413c80>] <class 'list'>


In [44]:
device = "cpu"
# device = "cuda"
# model = DPT(
#     seq_len=seq_len, input_dim=d, output_dim=d, hidden_dim=16, 
#     embedding_dropout=0.0, attention_dropout=0.0, residual_dropout=0.0
# ).to(device)
outputs = model(batch["x"].to(device), batch["y"].to(device))
outputs

tensor([[[-0.7146],
         [-0.1747],
         [-0.0399],
         [-0.1165]],

        [[-0.7146],
         [ 0.2069],
         [ 0.9116],
         [ 0.6276]]], grad_fn=<ViewBackward0>)

In [41]:
print('outputs', outputs.shape, outputs.dtype, outputs.device)

outputs torch.Size([2, 4, 1]) torch.float32 cpu
